In [27]:
from dataset1 import *
import numpy as np
from scipy.linalg import solve_triangular
from numpy.linalg import eigh
import torch
from hadamard_transform import hadamard_transform
import time
#test

## Define functions

In [128]:

def getError(A,B):
    return np.linalg.norm(A-B,ord='nuc')/np.linalg.norm(A,ord='nuc')

def fjlt(A, D, indices, direction):
    """
    Applies the Fast Johnson-Lindenstrauss Transform (FJLT) to a square matrix.

    Parameters:
    - A: Input square matrix (NumPy array of size n x m).
    - D: Diagonal vector of +1 and -1 (NumPy array of size n).
    - indices: Indices of the selected rows or columns for subsampling (NumPy array of size l).
    - direction: 'left' to apply FJLT from the left (on rows) or 'right' for the transpose case (on columns).

    Returns:
    - A_proj: Transformed matrix after applying FJLT and subsampling.
    """
    n, m = A.shape
    l = indices.shape[0]  # Sketch size
    if direction == 'left':  # FJLT applied to the rows of A
        assert len(D) == n, "The length of D must match the number of rows in A."
        assert (n & (n - 1) == 0), "The number of rows in A must be a power of 2."
        # Multiply each row of A by the corresponding element in D
        A_tilde = A * D[:, np.newaxis]
        # Apply the Fast Hadamard Transform to the rows
        A_fht = hadamard_transform(torch.tensor(A_tilde.T)).numpy().T
        # Subsample the rows and scale
        A_proj = A_fht[indices, :] * np.sqrt(n / l)

    elif direction == 'right':  # FJLT applied to the columns of A
        # Multiply each column of A by the corresponding element in D
        A_tilde = A * D[np.newaxis, :]
        # Apply the Fast Hadamard Transform to the columns
        A_fht = hadamard_transform(torch.tensor(A_tilde)).numpy()
        # Subsample the columns and scale
        A_proj = A_fht[:, indices] * np.sqrt(n / l)
    
    return A_proj

def getLowRank(A, rank, sketch_dim, sketch_method='gaussian', good_conditionned=True, print_times=False):
    """
    Compute a low-rank approximation of the matrix A using sketching, 
    eigenvalue decomposition, and truncation. Optionally prints the time taken for each major operation.

    Parameters:
        A (numpy.ndarray): Original matrix (size m x n).
        rank (int): Desired rank for the approximation.
        sketch_dim (int): Dimension of the sketching subspace.
        sketch_method (str): Method used for sketching. Options are:
                             - 'gaussian': Uses a Gaussian random matrix for sketching.
                             - 'fjlt': Uses the Fast Johnson-Lindenstrauss Transform.
        good_conditionned (bool): If True, uses Cholesky decomposition for better conditioning. 
                                  Otherwise, uses eigenvalue decomposition.
        print_times (bool): If True, prints the time taken for each major operation.

    Returns:
        A_nyst_k (numpy.ndarray): Low-rank approximation of A (size m x m).
    """
    m = A.shape[0]
    B = None
    C = None

    start_time = time.time()

    # Apply sketching based on the selected method
    if sketch_method == 'fjlt':  # Use Fast Johnson-Lindenstrauss Transform for sketching
        D = np.random.choice([1, -1], size=m)
        indices = np.random.choice(m, size=sketch_dim, replace=False)
        sketch_start = time.time()
        C = fjlt(A, D, indices, direction='right')
        B = fjlt(C, D, indices, direction='left')
        if print_times:
            print(f"FJLT sketching completed in {time.time() - sketch_start:.6f} seconds.")

    elif sketch_method == 'gaussian':  # Use Gaussian random matrix for sketching
        Omega = np.random.normal(size=(m, sketch_dim))
        sketch_start = time.time()
        C = A @ Omega
        B = Omega.T @ C
        if print_times:
            print(f"Gaussian sketching completed in {time.time() - sketch_start:.6f} seconds.")
    else:
        raise ValueError("Méthode de sketching non reconnue. Choisissez 'gaussian' ou 'fjlt'.")

    if good_conditionned:
        # Use Cholesky decomposition
        cholesky_start = time.time()
        L = np.linalg.cholesky(B)
        if print_times:
            print(f"Cholesky decomposition completed in {time.time() - cholesky_start:.6f} seconds.")
        
        triangular_start = time.time()
        Z = solve_triangular(L, C.T, lower=True).T
        if print_times:
            print(f"Triangular solve completed in {time.time() - triangular_start:.6f} seconds.")
    else:
        # Use eigenvalue decomposition
        eigh_start = time.time()
        eig_v, V = eigh(B)
        if print_times:
            print(f"Eigenvalue decomposition completed in {time.time() - eigh_start:.6f} seconds.")
        
        truncate_start = time.time()
        truncate = eig_v > 0
        eig_v = eig_v[truncate]
        V = V[:, truncate]
        Z = C @ (V * (1 / np.sqrt(eig_v)))
        if print_times:
            print(f"Eigenvalue truncation and normalization completed in {time.time() - truncate_start:.6f} seconds.")

    qr_start = time.time()
    Q, R = np.linalg.qr(Z)
    if print_times:
        print(f"QR decomposition completed in {time.time() - qr_start:.6f} seconds.")

    svd_start = time.time()
    U, Sigma, _ = np.linalg.svd(R)
    if print_times:
        print(f"SVD completed in {time.time() - svd_start:.6f} seconds.")

    approx_start = time.time()
    U_hat = Q @ U[:, :rank]
    A_nyst_k = (U_hat * Sigma[:rank]**2) @ U_hat.T
    if print_times:
        print(f"Low-rank approximation construction completed in {time.time() - approx_start:.6f} seconds.")

    total_time = time.time() - start_time
    if print_times:
        print(f"Total computation time: {total_time:.6f} seconds.")
    error = getError(A,A_nyst_k)
    print(f"Error :{error}".format(error))

## Tests

In [ ]:
n = 2048
A = getExpMatrix(n,R=10)

In [ ]:
getLowRank(A,rank=64,sketch_dim=128,sketch_method='gaussian',good_conditionned=False,print_times=True)

Gaussian sketching completed in 0.031635 seconds.
Eigenvalue decomposition completed in 0.000000 seconds.
Eigenvalue truncation and normalization completed in 0.011062 seconds.
QR decomposition completed in 0.004010 seconds.
SVD completed in 0.000000 seconds.
Low-rank approximation construction completed in 0.032002 seconds.
Total computation time: 0.096517 seconds.
Error :1.8213154841918373e-14


In [138]:
getLowRank(A,rank=64,sketch_dim=128,sketch_method='fjlt',good_conditionned=False,print_times=True)

FJLT sketching completed in 0.231160 seconds.
Eigenvalue decomposition completed in 0.000000 seconds.
Eigenvalue truncation and normalization completed in 0.019626 seconds.
QR decomposition completed in 0.009984 seconds.
SVD completed in 0.000000 seconds.
Low-rank approximation construction completed in 0.035765 seconds.
Total computation time: 0.298823 seconds.
Error :4.2085980039838814e-10
